Project Description

In [1]:
from sqlalchemy import create_engine
import pandas
import numpy
import seaborn
import matplotlib.pyplot as plt
%pylab inline

Populating the interactive namespace from numpy and matplotlib


Printing All Rows Function

In [ ]:
def print_full(column):
    pandas.set_option('display.max_rows', len(column))
    pandas.set_option('display.max_columns', 10000)
    print(column)
    pandas.reset_option('display.max_rows')
    pandas.reset_option('display.max_columns')

Moving MasterPBA Data from SQL Server into a pandas dataframe

In [ ]:
engine = create_engine('mssql://DW_PA')

MasterPBA = pandas.read_sql_query(
"""
SELECT
	jkg."bid id",
	jkg."LN ID",
	jkg."OriginMarket",
	jkg."DestMarket",
	jkg."AverageOTRMiles",
	jkg."AverageOTRRate",
	jkg."AverageHubRevenue",
	jkg."AverageIMLRate",
	jkg."AverageIMLMiles",
	jkg."O City",
	jkg."D City",
	jkg."Awarded Y/N",
	jkg."Target MPU",
	jkg."Dray Reduced",
	jkg."Floor Win %",
	jkg."Expert Win %",
	jkg."Expert MPU",
	jkg."Floor MPU",
	jkg."VOL",
	jkg."Total Rate",
	jkg."Original RPM",
	jkg."Target Win %",
	jkg."With Comp",
	jkg."With Truck",
	jkg."% With Comp",
	jkg."% With Truck",
	jkropf."O ST",
	jkropf."D ST",
	jkropf."EQ Type",
	jkropf."Miles",
	jkropf."O Ramp",
	jkropf."O MKT Ramp",
	jkropf."D Ramp",
	jkropf."D MKT Ramp",
	jkropf."PKU",
	jkropf."LHL",
	jkropf."DEL",
	jkropf."Lease/REZ",
	jkropf."Total Cost",
	jkropf."Fee",
	jkropf."Current TPS Rate",
	jkropf."Rate Status",
	jkropf."Hub Fuel Cost @ Analysis Level 1",
	jkropf."Cust Fuel @ Analysis Level 1",
	jkropf."Fuel Build In 1",
	jkropf."Cust Fuel Rate 1",
	jkropf."Cust Fuel Type 1 (PM/RP)",
	jkropf."DOE 1",
	jkropf."ODR Miles",
	jkropf."DDR Miles",
	jkropf."Haz Flag",
	jkropf."Price Cust #",
	jkropf."Awarded Volume",
	jkropf."Scope",
	jkropf."Segment",
	jkropf."Method Used",
	jkropf."Floor Margin %",
	jkropf."Target Margin %",
	jkropf."Expert Margin %",
	jkropf."Prior 6 mos Annualized",
	jkropf."fuelpct",
	jkropf."fuelnormed",
	jkropf."fndr"

FROM DW_PA.jkg.pba_uic as jkg JOIN DW_PA.jkropf.uicreplicated as jkropf
ON jkg."bid id" = jkropf."bid id"
AND jkg."LN ID" = jkropf. "LN ID";
"""
,engine)

Grabbing the DOE data from SQL Server into a pandas dataframe

In [ ]:
DOE = pandas.read_sql_query(
    """
    SELECT *
    FROM jkropf.hubimlfuel
    """
,engine)

Grabbing PBA Tracking info from SQL Server

In [ ]:
PBA_Tracking = pandas.read_sql_query(
    """
    SELECT
        "CUSTOMER_NAME",
        "CUSTOMER_NUMBER",
        "BID_ID",
        "DATE_AWARD_START",
        "Network",
        "IS_REPRICE",
        "CHANGE_VOLUME"
    FROM dbo.[MarketAnalysis_01_PBA Tracking]
    """
, engine)

Joining MasterPBA Data and PBA Tracking Info

In [ ]:
MasterPBA = pandas.merge(MasterPBA, PBA_Tracking, left_on="bid id", right_on="BID_ID", how="inner")
MasterPBA = MasterPBA.drop("BID_ID", axis=1)

Removing any DOE and Rate Value mismatches

In [ ]:
fuelpct_mismatch = []
for i in range(0,len(MasterPBA)):
    for j in range (0,len(DOE)):
        if MasterPBA.get_value(i,"DOE 1")>=DOE.get_value(j,"Min Amount") and MasterPBA.get_value(i,"DOE 1")<=DOE.get_value(j,"Max Amount"):
            if MasterPBA.get_value(i, "fuelpct") != DOE.get_value(j, "Rate Value"):
                fuelpct_mismatch.append(i)
MasterPBA = MasterPBA.drop(MasterPBA.index[[fuelpct_mismatch]])

Isolation to one instance of a lane within each bid

In [ ]:
MasterPBA = MasterPBA[pandas.notnull(MasterPBA["Fee"])]
MasterPBA = MasterPBA.drop_duplicates()
print MasterPBA.shape

Checking for any remaining duplicate lanes in MasterPBA

In [ ]:
Check = pandas.concat([MasterPBA["LN ID"], MasterPBA["bid id"]], axis=1)
Check["LNbid"] = Check["LN ID"] + Check["bid id"].map(str)
print_full(Check["LNbid"].value_counts())

After investigating further, the issue lanes for bid id 948854 have double spaces in the Fee column as opposed to a single space. The issue lanes for bid id 955884 have multiple lanes that are fee'd for a respective lane. The following code will deal with those issues.

In [ ]:
MasterPBA = MasterPBA[MasterPBA.Fee != " "]
MasterPBA = MasterPBA.reset_index(drop=True)
MasterPBA = MasterPBA.drop([3207, 3214, 3221, 3219])

Determining which lanes has had historical volume

In [ ]:
Past_Awarded = pandas.DataFrame(index=range(0,len(MasterPBA)), columns=["Past_Awarded"])
MasterPBA = pandas.concat([MasterPBA,Past_Awarded], axis=1)
for i in range(0,len(MasterPBA)):
    if pandas.isnull(MasterPBA.get_value(i, "Current TPS Rate")) == True:
        MasterPBA.set_value(i, "Past_Awarded", "No")
    else:
        MasterPBA.set_value(i, "Past_Awarded", "Yes")

Creating a Market Column, combining Origin Market and Destination Market
Creating a Ramp Column, combining O MKT Ramp and D MKT Ramp
Creating a Lane Key Column, combining O City, O ST, D City, D ST

In [ ]:
MasterPBA["Market"] = MasterPBA["OriginMarket"] + MasterPBA["DestMarket"]
MasterPBA["Ramp"] = MasterPBA["O MKT Ramp"] + MasterPBA["D MKT Ramp"]
MasterPBA["Lane"] = MasterPBA["O City"] + MasterPBA["O ST"] + MasterPBA["D City"] + MasterPBA["D ST"]

Grabbing Competitive Bid Stats for each Market with the most recent date

In [ ]:
Competitive = pandas.read_sql_query(
    """
    SELECT *
    FROM DW_PA.dbo.compPlot_stats
    WHERE runDt = (
        SELECT MAX(runDt)
        FROM DW_PA.dbo.compPlot_stats)
    """
,engine)

Creating a key for merging the Competetive Bid Stats with the MasterPBA Dataframe

In [ ]:
Competitive["Market"] = Competitive["OriginMarket"] + Competitive["DestMarket"]
Competitive = Competitive.drop(["OriginMarket", "DestMarket"], axis=1)

Removing any spaces from Market strings for both MasterPBA and Competitive

In [ ]:
MasterPBA["Market"] = MasterPBA["Market"].str.replace(" ", "")
Competitive["Market"] = Competitive["Market"].str.replace(" ", "")

Checking for any issues of collinearity within the competitive bid data and whether that will be an issue. It makes sense that AverageHubCost and AverageHubRevenue would be correlated variables. Null values in the StDevofCompRates, occurring when there is only one competitor.

In [ ]:
Competitive_Cor = Competitive.dropna()
seaborn.pairplot(Competitive_Cor)
seaborn.plt.show()

Joining the MasterPBA Data and Competitive Bid Data

In [ ]:
MasterPBA = pandas.merge(MasterPBA, Competitive, left_on="Market", right_on="Market", how="inner")
MasterPBA = MasterPBA.drop_duplicates()

Renaming AverageHubRevenue_y to AverageHubRevenue_Comp and removing AverageHubRevenue_x because the former is the most up to date version.

In [ ]:
MasterPBA = MasterPBA.rename(columns = {"AverageHubRevenue_y": "AverageHubRevenue_Comp"})
MasterPBA = MasterPBA.drop(["AverageHubRevenue_x"], axis=1)

Dropping any rows that has NaN for % With Truck and simultaneously is joined with Competitive information with Kind being Truck.

In [ ]:
MasterPBA = MasterPBA.drop(MasterPBA[(pandas.isnull(MasterPBA["% With Truck"]) == True) & (MasterPBA["Kind"] == "Truck")].index.values)
MasterPBA["% With Truck"].fillna(0, inplace=True)

Approximating AverageOTRMiles for missing data for instances in which trucks aren't used by averaging the state pairs and substituting those values as an approximation. Any unused state pairs are approximated with the use of Google Maps and are listed in the dictionary NA_STPAIRS.

In [ ]:
MasterPBA["STPAIR"] = MasterPBA["O ST"] + MasterPBA["D ST"]
AVG_OTR = pandas.DataFrame(index=range(0,len(MasterPBA["STPAIR"].unique())), columns=["AVG_OTR"])
a = pandas.concat([pandas.Series(MasterPBA["STPAIR"].unique()),AVG_OTR])
a = a.dropna(subset=[0])
for i in range(0,len(a)):
    a.set_value(i, "AVG_OTR", MasterPBA.groupby(["STPAIR"])["AverageOTRMiles"].mean()[i])
a.columns = ['STPAIR', 'AVG_OTR_STPAIR']

NA_STPAIRS = {'TXVA': 1403, 'NYCA': 2900, "UTNY": 2225, "ARNJ": 1183, "KSPA": 1269, "LANC": 947, "VTCA": 2650, "MNWA": 1464,
              "MAWA": 2944, "UTPA": 2045, "GAOR": 670, "GACO": 1624, "SCCO": 1672, "FLWY": 2118, "FLCO": 1966, "WIFL": 1391, "LAVA": 1014,
              "TXNY": 1830, "NVNC": 2477, "WAGA": 2719, "SDDE": 1518, "VACO": 1667, "MDWA": 2684, "IDSC": 2382, "WYVA": 1823, "UTKY": 1655,
              "UTMS": 1619, "IDAL": 2150, "OKMA": 1703, "IACT": 1168, "CAKS": 1539, "NVMO": 1643, "CAIA": 1848, "CANE": 1459, "MOTX": 727, 
              "TNUT": 1669, "MSUT": 1618, "GAWI": 1049, "MITX": 1378, "OHTX": 1285, "NCWA": 2759, "VAIN": 586, "NJCA": 2887, "BJCT": 2830,
              "WAAR": 2130, "WAKY": 2358, "WAAL": 2606, "MONC": 915, "NJOR":2761, "LACA": 1904, "AZMI": 1929, "METX":2222, "MALA": 1588,
              "NHTX": 1995, "RIAL": 1227, "IDMD": 2399, "GAWY": 1780, "TNWY": 1441, "MNME": 1437, "NDGA": 1751, "TXNH": 1996, "NMCT": 2073,
              "UTMA": 2406, "AZAL": 1595, "AZFL": 2148, "AZNY": 2380, "AZMA": 2559, "WANH": 2927, "NYID": 2476, "MAMN": 1538, "DEMN": 1345,
              "NYUT": 2228, "RIUT": 2391, "SCMN": 1384, "SCWI": 994, "ORMN": 1605}
for i in range(0,len(a)):
    if a.get_value(i, "STPAIR") in NA_STPAIRS.keys():
        a.set_value(i, "AVG_OTR_STPAIR", NA_STPAIRS[a.get_value(i, "STPAIR")])

MasterPBA = pandas.merge(MasterPBA,a, left_on="STPAIR", right_on="STPAIR", how="inner")
count = 0
for i in range(0,len(MasterPBA)):
    if pandas.isnull(MasterPBA.get_value(i, "AverageOTRMiles")) == True:
        MasterPBA.set_value(i, "AverageOTRMiles", MasterPBA.get_value(i, "AVG_OTR_STPAIR"))


The AverageOTRRate contains a lot of missing data as well, so the rates will be approximated using variables that are closely related. A simple OLS Regression will be used with the following variables as factors: AverageOTRMiles, AverageIMLRate, AverageIMLMiles, Market, Ramp, Lane, and Total Rate.

In [ ]:
AverageOTRRate = MasterPBA[["AverageOTRRate","AverageOTRMiles", "AverageIMLRate", "AverageIMLMiles", "Market", "Ramp", "Lane", "Total Rate"]]
AverageOTRRate_no_na = AverageOTRRate.dropna()
seaborn.pairplot(AverageOTRRate_no_na)
seaborn.plt.show()

In [ ]:
from sklearn import linear_model
from sklearn import preprocessing
from sklearn import cross_validation
from sklearn.metrics import mean_squared_error

numpy.random.seed(1337)

AverageOTRRate = MasterPBA[["AverageOTRRate","AverageOTRMiles", "AverageIMLRate", "AverageIMLMiles", "Market", "Ramp", "Lane", "Total Rate"]]

LE = preprocessing.LabelEncoder()
for i in range(0,len(AverageOTRRate.columns.values)):
    for i in range(0,len(AverageOTRRate.columns.values)):
        if AverageOTRRate.dtypes[i] == "object":
            temp = AverageOTRRate.columns.values[i]
            LE.fit(MasterPBA[temp].unique())
            AverageOTRRate[temp] = LE.transform(AverageOTRRate[temp])

NA_TEST = AverageOTRRate[pandas.isnull(AverageOTRRate["AverageOTRRate"]) == True]
AverageOTRRate = AverageOTRRate.dropna()

Target = AverageOTRRate["AverageOTRRate"]
AverageOTRRate = AverageOTRRate.drop(["AverageOTRRate"], axis=1)

x_train, x_test, y_train, y_test = cross_validation.train_test_split(AverageOTRRate, Target, test_size=.5)

LinearRegression = linear_model.LinearRegression()
LinearRegression.fit(x_train, y_train)
print 'MSE: ', mean_squared_error(y_test, LinearRegression.predict(x_test))

count = 0
for i in MasterPBA[pandas.isnull(MasterPBA["AverageOTRRate"]) == True].index.values:
    MasterPBA.set_value(i, "AverageOTRRate", LinearRegression.predict(NA_TEST.drop("AverageOTRRate", axis=1))[count])
    count += 1

Adjusting StDevofCompRates to be 0 for any lane that contains solely a single competitor

In [ ]:
for i in range (0, len(MasterPBA)):
    if (pandas.isnull(MasterPBA.get_value(i, "StDevofCompRates")) == True) & (MasterPBA.get_value(i, "NumberofComps") == 1):
        MasterPBA.set_value(i, "StDevofCompRates", 0)

In [ ]:
print_full(MasterPBA.isnull().sum())

Converting all Awarded Y/N's "N"s into "No"


In [ ]:
MasterPBA["Awarded Y/N"][MasterPBA["Awarded Y/N"] == "N"] = "No"
print MasterPBA["Awarded Y/N"].unique()

In [2]:
MasterPBA = pandas.read_csv('MasterPBA_Clean.csv')

C:\Users\john.wang\AppData\Local\Continuum\Anaconda2\lib\site-packages\IPython\core\interactiveshell.py:2723: DtypeWarning: Columns (30,42) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


Blending the Intermodal and Truck competitive rates to create an individual ramp that will reduce the data to only one awarded Y/N to each respective lane.

In [3]:
MasterPBA = MasterPBA.drop(["Unnamed: 0", "Unnamed: 0.1"], axis=1)
print MasterPBA.columns.values

['bid id' 'LN ID' 'OriginMarket' 'DestMarket' 'AverageOTRMiles'
 'AverageOTRRate' 'AverageIMLRate' 'AverageIMLMiles' 'O City' 'D City'
 'Awarded Y/N' 'Target MPU' 'Dray Reduced' 'Floor Win %' 'Expert Win %'
 'Expert MPU' 'Floor MPU' 'VOL' 'Total Rate' 'Original RPM' 'Target Win %'
 'With Comp' 'With Truck' '% With Comp' '% With Truck' 'O ST' 'D ST'
 'D Zip' 'EQ Type' 'Miles' 'O Ramp' 'O MKT Ramp' 'D Ramp' 'D MKT Ramp'
 'PKU' 'LHL' 'DEL' 'Lease/REZ' 'Total Cost' 'Fee' 'Current TPS Rate'
 'Rate Status' 'Hub Fuel Cost @ Analysis Level 1'
 'Cust Fuel @ Analysis Level 1' 'Fuel Build In 1' 'Cust Fuel Rate 1'
 'Cust Fuel Type 1 (PM/RP)' 'DOE 1' 'ODR Miles' 'DDR Miles' 'Haz Flag'
 'Price Cust #' 'Awarded Volume' 'Scope' 'Segment' 'Method Used'
 'Floor Margin %' 'Target Margin %' 'Expert Margin %'
 'Prior 6 mos Annualized' 'fuelpct' 'fuelnormed' 'fndr' 'CUSTOMER_NAME'
 'CUSTOMER_NUMBER' 'DATE_AWARD_START' 'Network' 'IS_REPRICE'
 'CHANGE_VOLUME' 'Past_Awarded' 'Market' 'Ramp' 'Lane' 'Kind'
 'Num

In [4]:
MasterPBA["BNumberofComps"] = 0.0
MasterPBA["BAverageCompRate"] = 0.0
MasterPBA["BStDevofCompRates"] = 0.0
MasterPBA["BAverageHubRevenue_Comp"] = 0.0
MasterPBA["BAverageHubCost"] = 0.0
MasterPBA["BAverageMiles"] = 0.0

for i in range(0, len(MasterPBA)):
    if MasterPBA.get_value(i, "Kind") == "Intermodal":
        MasterPBA.set_value(i, "BNumberofComps", MasterPBA.get_value(i, "NumberofComps"))
        MasterPBA.set_value(i, "BAverageCompRate", (1-MasterPBA.get_value(i, "% With Truck")/100.0)*MasterPBA.get_value(i, "AverageCompRate"))
        MasterPBA.set_value(i, "BStDevofCompRates", (1-MasterPBA.get_value(i, "% With Truck")/100.0)*MasterPBA.get_value(i, "StDevofCompRates"))
        MasterPBA.set_value(i, "BAverageHubRevenue_Comp", (1-MasterPBA.get_value(i, "% With Truck")/100.0)*MasterPBA.get_value(i, "AverageHubRevenue_Comp"))
        MasterPBA.set_value(i, "BAverageHubCost", (1-MasterPBA.get_value(i, "% With Truck")/100.0)*MasterPBA.get_value(i, "AverageHubCost"))
        MasterPBA.set_value(i, "BAverageMiles", (1-MasterPBA.get_value(i, "% With Truck")/100.0)*MasterPBA.get_value(i, "AverageMiles"))
    else:
        MasterPBA.set_value(i, "BNumberofComps", MasterPBA.get_value(i, "NumberofComps"))
        MasterPBA.set_value(i, "BAverageCompRate", (MasterPBA.get_value(i, "% With Truck")/100.0)*MasterPBA.get_value(i, "AverageCompRate"))
        MasterPBA.set_value(i, "BStDevofCompRates", (MasterPBA.get_value(i, "% With Truck")/100.0)*MasterPBA.get_value(i, "StDevofCompRates"))
        MasterPBA.set_value(i, "BAverageHubRevenue_Comp", (MasterPBA.get_value(i, "% With Truck")/100.0)*MasterPBA.get_value(i, "AverageHubRevenue_Comp"))
        MasterPBA.set_value(i, "BAverageHubCost", (MasterPBA.get_value(i, "% With Truck")/100.0)*MasterPBA.get_value(i, "AverageHubCost"))
        MasterPBA.set_value(i, "BAverageMiles", (MasterPBA.get_value(i, "% With Truck")/100.0)*MasterPBA.get_value(i, "AverageMiles"))

a = pandas.DataFrame(MasterPBA.groupby(["bid id", "LN ID"], as_index=False)["BNumberofComps", "BAverageCompRate","BStDevofCompRates","BAverageHubRevenue_Comp","BAverageHubCost", 
                                                                            "BAverageMiles"].sum())
MasterPBA = MasterPBA.drop(["BNumberofComps", "BAverageCompRate","BStDevofCompRates","BAverageHubRevenue_Comp","BAverageHubCost", "BAverageMiles"], axis=1)
MasterPBA = pandas.merge(MasterPBA, a, how='inner', on=['bid id', 'LN ID'])

MasterPBA = MasterPBA.drop(["Kind", "NumberofComps", "AverageCompRate","StDevofCompRates","AverageHubRevenue_Comp","AverageHubCost", "AverageMiles"], axis=1)


Correpsonding the names of the company with stocks, creating a categorical variable to determine whether they are publicly held companies or private.

In [ ]:
Stock = pandas.read_csv('Customer_Stock.csv')
MasterPBA = pandas.merge(MasterPBA, Stock, left_on="CUSTOMER_NAME", right_on="CUSTOMER_NAME", how="inner")

In [5]:
print MasterPBA.columns.values

['bid id' 'LN ID' 'OriginMarket' 'DestMarket' 'AverageOTRMiles'
 'AverageOTRRate' 'AverageIMLRate' 'AverageIMLMiles' 'O City' 'D City'
 'Awarded Y/N' 'Target MPU' 'Dray Reduced' 'Floor Win %' 'Expert Win %'
 'Expert MPU' 'Floor MPU' 'VOL' 'Total Rate' 'Original RPM' 'Target Win %'
 'With Comp' 'With Truck' '% With Comp' '% With Truck' 'O ST' 'D ST'
 'D Zip' 'EQ Type' 'Miles' 'O Ramp' 'O MKT Ramp' 'D Ramp' 'D MKT Ramp'
 'PKU' 'LHL' 'DEL' 'Lease/REZ' 'Total Cost' 'Fee' 'Current TPS Rate'
 'Rate Status' 'Hub Fuel Cost @ Analysis Level 1'
 'Cust Fuel @ Analysis Level 1' 'Fuel Build In 1' 'Cust Fuel Rate 1'
 'Cust Fuel Type 1 (PM/RP)' 'DOE 1' 'ODR Miles' 'DDR Miles' 'Haz Flag'
 'Price Cust #' 'Awarded Volume' 'Scope' 'Segment' 'Method Used'
 'Floor Margin %' 'Target Margin %' 'Expert Margin %'
 'Prior 6 mos Annualized' 'fuelpct' 'fuelnormed' 'fndr' 'CUSTOMER_NAME'
 'CUSTOMER_NUMBER' 'DATE_AWARD_START' 'Network' 'IS_REPRICE'
 'CHANGE_VOLUME' 'Past_Awarded' 'Market' 'Ramp' 'Lane' 'CorridorVol

In [ ]:
MasterPBA["Public"] = "cat"
for i in range(0,len(MasterPBA)):
    if MasterPBA.get_value(i, "Symbol") == "Private":
        MasterPBA.set_value(i, "Public","No")
    else:
        MasterPBA.set_value(i, "Public","Yes")

In [6]:
MasterPBA.to_csv('MasterPBA_Clean.csv')